In [1]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP, devuelve_patrones
from selenium import webdriver
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [2]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210311
dia = 11
mes = 03
año = 2021
11/03/2021


In [3]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [4]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
#pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']
pattern = devuelve_patrones()

# ------------------------------------------------------------------------

# DOGC

In [5]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [6]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8362']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Anuncios_de_la_Administración_de_justicia_']


In [7]:
#response

## Recoge Nombre Secciones Sumario

In [8]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [9]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia


## Recoge Valores para formar URLs Secciones Sumario

In [10]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Anuncios_de_la_Administración_de_justicia_


In [11]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [12]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [13]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [14]:
df_sumarios['URL_Seccion'][0]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Disposiciones'

## Recoge Items en Seccion Disposiciones

In [15]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [16]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [17]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Disposiciones


In [18]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [19]:
URL_HTML_disposiciones

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8362#section-Disposiciones'

In [20]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de la Presidencia', 'Departamento de la Vicepresidencia y de Economía y Hacienda', 'Departamento de Educación', 'Departamento de Salud', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Cultura', 'Departamento de Empresa y Conocimiento', 'Departamento de Agricultura, Ganadería, Pesca y Alimentación']
['RESOLUCIÓN PRE/656/2021, de 7 de marzo, por la que se abre la convocatoria para otorgar compensaciones económicas a favor de los ayuntamientos para que abonen retribuciones a determinados cargos electos locales para el ejercicio 2021 (ref. BDNS 552459).', 'RESOLUCIÓN VEH/668/2021, de 5 de marzo, por la que se da publicidad al Convenio único de colaboración entre la Agencia Tributaria de Cataluña y el Ayuntamiento de Reus en materia tributaria.', 'DECRETO 16/2021, de 9 de marzo, por el que se crea el título de la Generalidad de técnico o técnica en Técnicas de Danza Urbana, y se establece su currículum.', 'ACUERDO GOV/25/2021, de 9 de marzo,  por el que se ap

In [21]:
len(departamentos)

8

In [22]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [23]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [24]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [25]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,"RESOLUCIÓN PRE/656/2021, de 7 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"RESOLUCIÓN VEH/668/2021, de 5 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"DECRETO 16/2021, de 9 de marzo, por el que se ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"ACUERDO GOV/25/2021, de 9 de marzo, por el qu...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"ACUERDO GOV/26/2021, de 9 de marzo, por el que...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,"RESOLUCIÓN TES/674/2021, de 9 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"RESOLUCIÓN TES/642/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,"RESOLUCIÓN CLT/635/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,"RESOLUCIÓN CLT/636/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,"RESOLUCIÓN CLT/637/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [26]:
DOGC_sumarios['item_urlHTML'][1]

'https://dogc.gencat.cat/es/document-del-dogc/?documentId=895362'

In [27]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(3)

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

In [28]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC
0,"RESOLUCIÓN PRE/656/2021, de 7 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Decreto 69/2008, de 1 de abril, de regulaci...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[07/03/2021],[PRE/0656/2021],[21068043],[Departamento de la Presidencia],[CVE-DOGC-B-21068043-2021],[8362],[11/03/2021],[Disposiciones]
1,"RESOLUCIÓN VEH/668/2021, de 5 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,En fecha 31 de julio de 2020 se ha firmado el ...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[05/03/2021],[VEH/0668/2021],[21068044],[Agencia Tributaria de Cataluña],[CVE-DOGC-B-21068044-2021],[8362],[11/03/2021],[Disposiciones]
2,"DECRETO 16/2021, de 9 de marzo, por el que se ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"En materia de enseñanza no universitaria, el a...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Decreto],[09/03/2021],[016/2021],[21068120],[Departamento de Educación],[CVE-DOGC-B-21068120-2021],[8362],[11/03/2021],[Disposiciones]
3,"ACUERDO GOV/25/2021, de 9 de marzo, por el qu...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El artículo 7.2.4 de la Ley 15/1990, de 9 de j...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Acuerdo del gobierno ],[09/03/2021],[GOV/0025/2021],[21069006],[Departamento de Salud],[CVE-DOGC-B-21069006-2021],[8362],[11/03/2021],[Disposiciones]
4,"ACUERDO GOV/26/2021, de 9 de marzo, por el que...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La estrategia de respuesta de las autoridades ...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Acuerdo del gobierno ],[09/03/2021],[GOV/0026/2021],[21069007],[Departamento de Salud],[CVE-DOGC-B-21069007-2021],[8362],[11/03/2021],[Disposiciones]
5,"RESOLUCIÓN TES/674/2021, de 9 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Visto que la Ponencia Ambiental, en la sesión ...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[09/03/2021],[TES/0674/2021],[21068047],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21068047-2021],[8362],[11/03/2021],[Disposiciones]
6,"RESOLUCIÓN TES/642/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El artículo 3.1.g) del texto refundido de la l...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[TES/0642/2021],[21067039],[Agencia Catalana del Agua],[CVE-DOGC-B-21067039-2021],[8362],[11/03/2021],[Disposiciones]
7,"RESOLUCIÓN CLT/635/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[CLT/0635/2021],[21067042],[Oficina de Apoyo a la Iniciativa Cultural],[CVE-DOGC-B-21067042-2021],[8362],[11/03/2021],[Disposiciones]
8,"RESOLUCIÓN CLT/636/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[CLT/0636/2021],[21067043],[Oficina de Apoyo a la Iniciativa Cultural],[CVE-DOGC-B-21067043-2021],[8362],[11/03/2021],[Disposiciones]
9,"RESOLUCIÓN CLT/637/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consej

In [29]:
DOGC_sumarios['item_fullText'][0]

'El Decreto 69/2008, de 1 de abril, de regulación de un sistema de compensaciones económicas a favor de los ayuntamientos para que abonen retribuciones a determinados cargos electos locales, establece el régimen jurídico de atribución de un sistema de compensaciones económicas a favor de los ayuntamientos de hasta 2.000 habitantes para que abonen retribuciones a determinados cargos electos locales y dispone que cada año se publicará una convocatoria en la que se indicará la compensación máxima anual a percibir por cada ayuntamiento solicitante, de acuerdo con el número de habitantes inscritos en el padrón, con los tramos de población y otros requisitos regulados en el Decreto.\nDe acuerdo con el artículo 51 de la Ley 4/2020, de 29 de abril, de presupuestos de la Generalidad de Cataluña para 2020, y con el Decreto 146/2020, de 15 de diciembre, por el que se establecen los criterios de aplicación de la prórroga de los presupuestos de la Generalidad de Cataluña para 2020, mientras no entr

In [30]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [31]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [32]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

['Ley 4/2020', 'Ley 4/2020', 'Ley 38/2003', 'Ley 18/2010', 'Ley 39/2015', 'Ley 19/2014', 'Ley 29/1998', 'Orden ECO/172/2015', 'Orden VEH/79/2020', 'Ley 38/2003', 'Ley 29/1998', 'Ley 29/1998', 'Resolución PRE/1384/2019']
['Ley 26/2010', 'Ley 40/2015', 'Ley 17/2017', 'Real Decreto Legislativo 2/2004', 'Ley 17/2017', 'Resolución ECO/184/2014', 'Real Decreto Legislativo 2/2004', 'Ley 40/2015', 'Ley 26/2010', 'Ley 40/2015', 'Ley 40/2015', 'Ley 7/1985', 'Real Decreto Legislativo 2/2004', 'Real Decreto 939/2005', 'Ley 58/2003', 'Reglamento (UE) 2016/679', 'Ley 40/2015', 'Ley 40/2015', 'Ley 58/2003', 'Reglamento (UE) 2016/679', 'Ley 26/2010', 'Ley 40/2015', 'Ley 26/2010', 'Ley 9/2017', 'Ley 17/2017', 'Ley 7/1985', 'Real Decreto Legislativo 2/2004', 'Ley 58/2003', 'Ley 39/2015', 'Ley 29/2010', 'Reglamento (UE) 2016/679', 'Ley 26/2010', 'Ley 29/1998', 'Ley 40/2015', 'Ley 26/2010', 'Ley 19/2014', 'Ley 39/2015', 'Resolución VEH/3102/2019', 'Ley 17/2017', 'Ley 58/2003', 'Reglamento (UE) 2016/679']


In [33]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

['Ley 4/2020', 'Ley 19/2014', 'Orden ECO/172/2015', 'Orden VEH/79/2020', 'Ley 38/2003', 'Ley 18/2010', 'Ley 39/2015', 'Ley 29/1998', 'Resolución PRE/1384/2019']
['Ley 58/2003', 'Ley 40/2015', 'Ley 19/2014', 'Reglamento (UE) 2016/679', 'Resolución ECO/184/2014', 'Ley 7/1985', 'Ley 26/2010', 'Real Decreto 939/2005', 'Ley 39/2015', 'Ley 29/1998', 'Resolución VEH/3102/2019', 'Ley 17/2017', 'Real Decreto Legislativo 2/2004', 'Ley 9/2017', 'Ley 29/2010']
['Ley 19/2014', 'Ley 13/2008', 'Ley 12/2009', 'Ley 26/2010']
['Resolución de 30 de marzo de 1995', 'Ley 13/2008', 'Ley 9/2017', 'Ley 15/1990']
[]
['Ley 22/1973', 'Ley 21/2013', 'Ley 20/2009', 'Ley 12/1981', 'Ley 22/2011', 'Ley 16/2002']
['Resolución TES/2921/2020', 'Ley 39/2015', 'Ley 29/1988']
['Resolución CLT/956/2020', 'Orden CLT/4/2013', 'Resolución CLT/845/2019', 'Resolución CLT/198/2021', 'Orden CLT/47/2014', 'Resolución CLT/3216/2020']
['Resolución CLT/956/2020', 'Orden CLT/4/2013', 'Ley 39/2015', 'Resolución CLT/845/2019', 'Resolució

In [34]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [35]:
print(regex_result)

['Ley 4/2020', 'Ley 19/2014', 'Orden ECO/172/2015', 'Orden VEH/79/2020', 'Ley 38/2003', 'Ley 18/2010', 'Ley 39/2015', 'Ley 29/1998', 'Resolución PRE/1384/2019', 'Ley 58/2003', 'Ley 40/2015', 'Ley 19/2014', 'Reglamento (UE) 2016/679', 'Resolución ECO/184/2014', 'Ley 7/1985', 'Ley 26/2010', 'Real Decreto 939/2005', 'Ley 39/2015', 'Ley 29/1998', 'Resolución VEH/3102/2019', 'Ley 17/2017', 'Real Decreto Legislativo 2/2004', 'Ley 9/2017', 'Ley 29/2010', 'Ley 19/2014', 'Ley 13/2008', 'Ley 12/2009', 'Ley 26/2010', 'Resolución de 30 de marzo de 1995', 'Ley 13/2008', 'Ley 9/2017', 'Ley 15/1990', 'Ley 22/1973', 'Ley 21/2013', 'Ley 20/2009', 'Ley 12/1981', 'Ley 22/2011', 'Ley 16/2002', 'Resolución TES/2921/2020', 'Ley 39/2015', 'Ley 29/1988', 'Resolución CLT/956/2020', 'Orden CLT/4/2013', 'Resolución CLT/845/2019', 'Resolución CLT/198/2021', 'Orden CLT/47/2014', 'Resolución CLT/3216/2020', 'Resolución CLT/956/2020', 'Orden CLT/4/2013', 'Ley 39/2015', 'Resolución CLT/845/2019', 'Resolución CLT/198/

In [36]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [37]:
boletin_flat_list

['Ley 58/2003',
 'Ley 22/1973',
 'Ley 4/2020',
 'Ley 4/2008',
 'Resolución ECO/184/2014',
 'Ley 7/1985',
 'Resolución CLT/187/2021',
 'Resolución CLT/204/2021',
 'Ley 13/2014',
 'Ley 13/2008',
 'Real decreto legislativo 5/2000',
 'Real decreto 887/2006',
 'Ley 5/2008',
 'Ley 15/1990',
 'Resolución CLT/613/2018',
 'Real Decreto Legislativo 2/2004',
 'Resolución PRE/1384/2019',
 'Ley 12/2009',
 'Resolución CLT/3216/2020',
 'Real decreto 39/1997',
 'Ley 29/1988',
 'Orden ECO/172/2015',
 'Reglamento (UE) 651/2014',
 'Orden VEH/79/2020',
 'Resolución EMC/350/2021',
 'Real Decreto 939/2005',
 'Ley 21/2013',
 'Ley 20/2000',
 'Ley 38/2003',
 'Resolución de 30 de marzo de 1995',
 'Ley 1/1998',
 'Ley 29/1998',
 'Ley 31/1995',
 'Orden ARP/121/2017',
 'Ley 17/2017',
 'Ley 12/1981',
 'Ley 22/2011',
 'Reglamento (UE) 972/2020',
 'Resolución CLT/198/2021',
 'Ley 9/2017',
 'Orden CLT/47/2014',
 'Resolución EMC/2810/2020',
 'Ley 40/2015',
 'Orden CLT/4/2013',
 'Ley 20/2009',
 'Reglamento (UE) 702/2014'

In [38]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


['España', 'Europa', 'Cataluña']


In [39]:
#boletin_ASECORP_flat_list

In [40]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

set()

In [41]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 9/2017'}
{'Ley 9/2017'}
{'Ley 20/2009'}
{'Ley 16/2002'}
{'Ley 12/1981'}
{'Ley 12/1981'}
{'Ley 16/2002'}
{'Ley 16/2002'}
{'Ley 20/2009'}
{'Ley 21/2013'}
{'Ley 22/1973'}
{'Ley 22/2011'}
{'Ley 22/2011', 'Ley 16/2002'}
{'Ley 22/1973'}
{'Ley 21/2013'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 20/2009'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 22/2011'}
{'Ley 16/2002'}
{'Ley 16/2002'}
{'Ley 21/2013'}
{'Ley 20/2009'}
{'Ley 20/2009'}
{'Ley 20/2009'}
{'Ley 20/2009'}
{'Ley 21/2013'}
{'Ley 21/2013'}
{'Ley 13/2014'}
{'Ley 31/1995'}
{'Ley 54/2003'}
{'Ley 5/2008'}
{'Ley 31/1995'}
{'Ley 31/1995'}
{'Ley 31/1995'}


In [42]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 9/2017'} LEY 9/2017 ESP
{'Ley 9/2017'} LEY 9/2017 ESP
{'Ley 20/2009'} BARCELONA 12/1/2011
{'Ley 16/2002'} DEC 176/2009 CAT
{'Ley 12/1981'} DLG 14/1994 CAT
{'Ley 12/1981'} LEY 12/1981 CAT
{'Ley 16/2002'} LEY 16/2002 CAT
{'Ley 16/2002'} LEY 16/2002 ESP
{'Ley 20/2009'} LEY 20/2009 CAT
{'Ley 21/2013'} LEY 21/2013 ESP
{'Ley 22/1973'} LEY 22/1973 ESP
{'Ley 22/2011'} LEY 22/2011 ESP
{'Ley 22/2011', 'Ley 16/2002'} LEY 5/2013 ESP
{'Ley 22/1973'} LEY 54/1980 ESP
{'Ley 21/2013'} LEY 9/2018 ESP
{'Ley 22/2011'} ORD AAA/699/2016 ESP
{'Ley 22/2011'} ORD APM/189/2018 ESP
{'Ley 22/2011'} ORD APM/205/2018 ESP
{'Ley 22/2011'} ORD APM/206/2018 ESP
{'Ley 22/2011'} ORD APM/397/2018 ESP
{'Ley 20/2009'} ORD MAH/611/2010 CAT
{'Ley 22/2011'} ORD TEC/852/2019 ESP
{'Ley 22/2011'} ORD TED/363/2020 ESP
{'Ley 22/2011'} ORD TED/426/2020 ESP
{'Ley 16/2002'} RD 509/2007 ESP
{'Ley 16/2002'} RD 815/2013 ESP
{'Ley 21/2013'} RES 8/3/2016 ESP
{'Ley 20/2009'} RES TES/1007/2014 CAT
{'Ley 20/2009'} RES TES/2243/2013 CAT


In [43]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,Tags,Match_ASECORP_BBDD
0,"RESOLUCIÓN PRE/656/2021, de 7 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Decreto 69/2008, de 1 de abril, de regulaci...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[07/03/2021],[PRE/0656/2021],[21068043],[Departamento de la Presidencia],[CVE-DOGC-B-21068043-2021],[8362],[11/03/2021],[Disposiciones],"[Ley 4/2020, Ley 19/2014, Orden ECO/172/2015, ...",[]
1,"RESOLUCIÓN VEH/668/2021, de 5 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,En fecha 31 de julio de 2020 se ha firmado el ...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[05/03/2021],[VEH/0668/2021],[21068044],[Agencia Tributaria de Cataluña],[CVE-DOGC-B-21068044-2021],[8362],[11/03/2021],[Disposiciones],"[Ley 58/2003, Ley 40/2015, Ley 19/2014, Reglam...",[LEY 9/2017 ESP]
2,"DECRETO 16/2021, de 9 de marzo, por el que se ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"En materia de enseñanza no universitaria, el a...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Decreto],[09/03/2021],[016/2021],[21068120],[Departamento de Educación],[CVE-DOGC-B-21068120-2021],[8362],[11/03/2021],[Disposiciones],"[Ley 19/2014, Ley 13/2008, Ley 12/2009, Ley 26...",[]
3,"ACUERDO GOV/25/2021, de 9 de marzo, por el qu...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El artículo 7.2.4 de la Ley 15/1990, de 9 de j...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Acuerdo del gobierno ],[09/03/2021],[GOV/0025/2021],[21069006],[Departamento de Salud],[CVE-DOGC-B-21069006-2021],[8362],[11/03/2021],[Disposiciones],"[Resolución de 30 de marzo de 1995, Ley 13/200...",[LEY 9/2017 ESP]
4,"ACUERDO GOV/26/2021, de 9 de marzo, por el que...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La estrategia de respuesta de las autoridades ...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Acuerdo del gobierno ],[09/03/2021],[GOV/0026/2021],[21069007],[Departamento de Salud],[CVE-DOGC-B-21069007-2021],[8362],[11/03/2021],[Disposiciones],[],[]
5,"RESOLUCIÓN TES/674/2021, de 9 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Visto que la Ponencia Ambiental, en la sesión ...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[09/03/2021],[TES/0674/2021],[21068047],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21068047-2021],[8362],[11/03/2021],[Disposiciones],"[Ley 22/1973, Ley 21/2013, Ley 20/2009, Ley 12...","[BARCELONA 12/1/2011, DEC 176/2009 CAT, DLG 14..."
6,"RESOLUCIÓN TES/642/2021, de 4 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El artículo 3.1.g) del texto refundido de la l...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[04/03/2021],[TES/0642/2021],[21067039],[Agencia Catalana del Agua],[CVE-DOGC-B-21067039-2021],[8362],[11/03/2021],[Disposiciones],"[Resolución TES/2921/2020, Ley 39/2015, Ley 29...",[]
7,"RESOLUCIÓN CLT/635/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[02/03/2021],[CLT/0635/2021],[21067042],[Oficina de Apoyo a la Iniciativa Cultural],[CVE-DOGC-B-21067042-2021],[8362],[11/03/2021],[Disposiciones],"[Resolución CLT/956/2020, Orden CLT/4/2013, Re...",[]
8,"RESOLUCIÓN CLT/636/2021, de 2 de marzo, por la...",https://dogc.gencat.cat/es/document-del-dogc/?...

# Genera Fichero EXCEL de resultados

In [44]:
DOGC_sumarios_final = DOGC_sumarios

In [45]:
DOGC_sumarios_final.dtypes

item_Title            object
item_urlHTML          object
item_urlPDF           object
item_fullText         object
item_urlXML           object
tipo_doc              object
fecha_doc             object
numero_doc            object
numero_control        object
entidad_emisora       object
CVE                   object
numero_DOGC           object
fecha_DOGC            object
seccion_DOGC          object
Tags                  object
Match_ASECORP_BBDD    object
dtype: object

In [46]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [47]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [48]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [49]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [50]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [51]:
DOGC_sumarios_final_CSV['item_id'][0]

'=HIPERVINCULO("https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=895401&type=01&language=es_ES";"CVE-DOGC-B-21068043-2021")'

In [52]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [53]:
DOGC_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,CVE-DOGC-B-21068043-2021,"RESOLUCIÓN PRE/656/2021, de 7 de marzo, por la...",11/03/2021,"Ley 4/2020, Ley 19/2014, Orden ECO/172/2015, O...",
1,CVE-DOGC-B-21068044-2021,"RESOLUCIÓN VEH/668/2021, de 5 de marzo, por la...",11/03/2021,"Ley 58/2003, Ley 40/2015, Ley 19/2014, Reglame...",LEY 9/2017 ESP
2,CVE-DOGC-B-21068120-2021,"DECRETO 16/2021, de 9 de marzo, por el que se ...",11/03/2021,"Ley 19/2014, Ley 13/2008, Ley 12/2009, Ley 26/...",
3,CVE-DOGC-B-21069006-2021,"ACUERDO GOV/25/2021, de 9 de marzo, por el qu...",11/03/2021,"Resolución de 30 de marzo de 1995, Ley 13/2008...",LEY 9/2017 ESP
4,CVE-DOGC-B-21069007-2021,"ACUERDO GOV/26/2021, de 9 de marzo, por el que...",11/03/2021,,
5,CVE-DOGC-B-21068047-2021,"RESOLUCIÓN TES/674/2021, de 9 de marzo, por la...",11/03/2021,"Ley 22/1973, Ley 21/2013, Ley 20/2009, Ley 12/...","BARCELONA 12/1/2011, DEC 176/2009 CAT, DLG 14/..."
6,CVE-DOGC-B-21067039-2021,"RESOLUCIÓN TES/642/2021, de 4 de marzo, por la...",11/03/2021,"Resolución TES/2921/2020, Ley 39/2015, Ley 29/...",
7,CVE-DOGC-B-21067042-2021,"RESOLUCIÓN CLT/635/2021, de 2 de marzo, por la...",11/03/2021,"Resolución CLT/956/2020, Orden CLT/4/2013, Res...",
8,CVE-DOGC-B-21067043-2021,"RESOLUCIÓN CLT/636/2021, de 2 de marzo, por la...",11/03/2021,"Resolución CLT/956/2020, Orden CLT/4/2013, Ley...",
9,CVE-DOGC-B-21067044-2021,"RESOLUCIÓN CLT/637/2021, de 2 de marzo, por la...",11/03/2021,"Orden CLT/4/2013, Resolución CLT/613/2018, Res...",


In [54]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 